In [1]:
# load credentials
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [2]:
# load modules
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
import os
from langchain_community.document_loaders import TextLoader

folder_path = "web_scraping"

# Get all .txt files in the folder and limit to first 100
txt_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')][:10]


all_pages = []  # To store pages from all files

for txt in txt_files:
    # print(f"Processing {txt} file...")  # Optional: Show progress
    loader = TextLoader(file_path=os.path.join(folder_path, txt), encoding="utf-8")
    
    # Load the documents for this file and append to `all_pages`
    for doc in loader.lazy_load():
        all_pages.append(doc)

# Output all collected pages
print(f"✅ Total pages loaded: {len(all_pages)}")


✅ Total pages loaded: 10


In [4]:
# Splitting document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 200,
        separators=[
        "\n"]
)

splits = text_splitter.split_documents(all_pages)
# splits

In [5]:
len(splits)

67

# Embedding model: BAAI/bge-m3

In [6]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

hf_embedding_model = HuggingFaceInferenceAPIEmbeddings(
    model_name="BAAI/bge-m3",
    api_key=HF_TOKEN,
)

print("embedding model loaded")

embedding model loaded


In [7]:
from astrapy import DataAPIClient

ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
# ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")

# Initialize the client
client = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN)
db = client.get_database_by_api_endpoint(
  ASTRA_DB_API_ENDPOINT
)

print(f"Connected Database: {db.info().name}\nCollections found: {db.list_collection_names()}")

Connected Database: research_paper_bot
Collections found: ['pregnancy_bot', 'travel_bot']


In [8]:
# create collection
from langchain_astradb import AstraDBVectorStore

vector_store = AstraDBVectorStore(
    collection_name="travel_bot",
    embedding=hf_embedding_model,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace="default_keyspace",
    # autodetect_collection=True,   # set to True to use it
)

vector_store.collection_name

APICommander about to raise from: [{'message': "Collection already exists: trying to create Collection ('travel_bot') with different settings", 'errorCode': 'EXISTING_COLLECTION_DIFFERENT_SETTINGS', 'id': '2690c2e4-1c40-4e08-b4da-d1e0178dd3bf', 'title': 'Collection already exists', 'family': 'REQUEST', 'scope': 'EMPTY'}]
c:\Users\ASUS\miniconda3\envs\torch\Lib\site-packages\langchain_astradb\utils\astradb.py:391: UserWarning: Astra DB collection 'travel_bot' is detected as having indexing turned on for all fields (either created manually or by older versions of this plugin). This implies stricter limitations on the amount of text each string in a document can store. Consider indexing anew on a fresh collection to be able to store longer texts. See https://github.com/langchain-ai/langchain-datastax/blob/main/libs/astradb/README.md#warnings-about-indexing for more details.
  if not self._validate_indexing_policy(


'travel_bot'

In [9]:
vector_store.add_documents(documents=splits)

['b44f4b08c735471092d197b682fd69ad',
 '2b431eb880504e3787a4b4cadef97066',
 '3b86faf3295245b48aeafb05c024cbfb',
 '410ebeb93ca149e19337f21be4712f14',
 '0faf8496126e47238a7daf3f843e6fa8',
 '223ecdc85bb14a1587df4627d93e1bc0',
 '37720a7d991c44a2bc810e36893fa609',
 'd38844e0934c41a590358edd5f045c3f',
 '198db8c581f7429e83ef8b9055a067c0',
 'c740ce72099b4d9b81751a653a861c22',
 '9182da95976b4d28bb5346ebd6a2c311',
 '0bfc992e17e9413e9cec9aa91f5d67c4',
 'be3f574daae84d3cae6dae00e33f5ee8',
 '228c7ca228894b02b8901ccfe5295fb5',
 '1d4cd9852d844c43a029128b6ec5f010',
 '2c7bb072b7aa48e8bd6203105ccf783e',
 '1411feea8c6b4d8e9cd85a39cfae620f',
 '9175a46ec68642c88f80f701036e718c',
 '49be44565d944195ab7e766be4e1e186',
 'c2d21203872d45f8b7c415f25f516971',
 '4b1b3e1831d94d01bf4c968007cf7c5e',
 '70b6c549d4bb4524aad5914352aec021',
 'bf2f1b4a824444a2a2fc6542d8df02fc',
 'd4d139ad4ce9491e9cf42dd41982c1bb',
 '3e4be5b3203a434a891534f57a8f1d11',
 '00f6437a2625477dbfb9398f04cc6106',
 '0affcdbc6a384a19b35be55e02ef6847',
 

In [12]:
# as data is already loaded

vector_store = AstraDBVectorStore(
    embedding = hf_embedding_model,
    collection_name="travel_bot",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace="default_keyspace",
    autodetect_collection=True,
)

## Vector Retriever

In [13]:
vec_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [14]:
# sanity check for similarity search
results = vector_store.similarity_search_with_score(
    "নীল কুঠি কোথায়?",
    k=2,
)
for res, score in results:
    print(f"*** [Similarity score={score:3f}]\n{res.page_content}\n[{res.metadata}]\n")

*** [Similarity score=0.777963]
প্রায় দুইশত বছর আগে ডানলপ নামের এক ইংরেজ নীলকর ১২ একর জমির ওপর এই নীলকুঠিরটি স্থাপন করেন। তখন কৃষকদের অন্য ফসল ছেড়ে নীল চাষে বাধ্য করা হত। এক সময় নীল কুঠিয়াল ও তাদের দোসর জমিদার-মহাজনদের অত্যাচারে চরম আকার ধারন করে। তখন শিবচর উপজেলার বাহাদুরপুর গ্রামের হাজী শরীয়তউল্লাহ ও তার পুত্র পীর মহসীনউদ্দিন দুদুমিয়া নীল কুঠিয়ালদের বিরুদ্ধে প্রতিরোধ গড়ে তোলেন। কৃষকদের অধিকার রক্ষার এই প্রতিবাদ এক সময় গণআন্দোলনে পরিনত হয়।
[{'source': 'web_scraping\\আউলিয়াপুর নীলকুঠি.txt'}]

*** [Similarity score=0.768082]

ঐতিহাসিক ফরায়েজী আন্দোলনের সাক্ষী আউলিয়াপুর নীলকুঠি (Auliapur Nilkuthi) মাদারীপুর অঞ্চলের মানুষের কাছে ডানলপ সাহেবের নীলকুঠি (Madaripur Dunlop Nilkuthi) নামেও পরিচিত। আউলিয়াপুর নীলকুঠি যেন বর্তমানেও গরীব চাষীদের উপর ব্রিটিশ নীলকরদের শোষন এবং তৎকালীন সময়কে বহন করে চলেছে। আউলিয়াপুর নীলকুঠিরটি ছিলারচর ইউনিয়নের আউলিয়াপুর গ্রামে অবস্থিত। মাদারীপুর জেলা শহর হতে এই নীলকুঠিরের দূরত্ব মাত্র ১০ কিলোমিটার। পীর-আউলিয়ার পদস্পর্শে ধন্য আউলিয়াপুরের খ্যাতিমান আউলিয়া হ